In [ ]:

import os,sys
sys.path.append('/users/danieljordan/DEEPlasma/')
import matplotlib.pyplot as plt
import numpy as np
import os,sys
from scipy.fft import fft, fftfreq
from GENE_ML.gene_ml.parsers.GENEparser import GENE_scan_parser
from GENE_ML.gene_ml.parsers.parser_timeseries import ParserTimeseries
from GENE_ML.gene_ml.runners.calculate_growthrate import calculate_growthrate
from GENE_ML.gene_ml.runners.calculate_growthrate import calculate_growthrate2

parser = GENE_scan_parser()
parser_ts = ParserTimeseries()

def plot_growthrate_history(scanfiles_dir, suffix, title='', tlim=None, converged_value=None, xlim=[None, None], ylim=[None,None]):
    energy_path = os.path.join(scanfiles_dir,f'energy_{suffix}')
    nrg_path = os.path.join(scanfiles_dir,f'nrg_{suffix}')
    time_units_to_average = 10
    # fig = plt.figure()
    
    if nrg_path != None:
        Qes, time_qes = parser_ts.Qes_history(nrg_path)
        eQes = Qes['1']
        time_in_1_index_shift = time_qes[-1] / len(time_qes)
        index_shift = int(time_units_to_average/time_in_1_index_shift)
        Qes_growthrate = np.empty(0)
        Q_gr_time = np.empty(0)
        # plt.figure()
        # plt.plot(time_qes, np.log(eQes))
        # plt.xlim(*xlim)
        for i in range(index_shift,len(time_qes)):
            Qrange = np.log(eQes[i-index_shift:i])
            range_t = time_qes[i-index_shift:i]
            coeff = np.polyfit(range_t, Qrange, 1)
            Qes_gr = coeff[0]/2
            Qes_growthrate = np.append(Qes_growthrate,Qes_gr)
            Q_gr_time = np.append(Q_gr_time, time_qes[i])
        #     plt.plot(range_t, Qes_gr*range_t + coeff[1])
    plt.figure()
    plt.title(title)
    print(Qes_growthrate)
    plt.plot(Q_gr_time, Qes_growthrate, 'x', label='eQes growthrate', color='magenta')

    

    growthrate1, time1 = calculate_growthrate2(scanfiles_dir, suffix)
    growthrate, time = parser_ts.growthrate_history(energy_path)
    #s = i*120
    if tlim != None:
        tdif = np.abs(time - tlim)
        tdif1 = np.abs(time1 - tlim)
        t = time[0:np.argmin(tdif)]#[s:s+120]
        t1 = time1[0:np.argmin(tdif1)]#[s:s+120]
        gr = growthrate[0:np.argmin(tdif)]#[s:s+120]
        gr1 = growthrate1[0:np.argmin(tdif1)]#[s:s+120]
        
    else:
        t = time
        t1 = time1
        gr = growthrate
        gr1 = growthrate1
    
    plt.ylabel('Growthrate')
    plt.plot(t, gr, label='Tobias Growthrate')
    time_in_1_index_shift = t[-1] / len(t)
    index_shift = int(time_units_to_average/time_in_1_index_shift)
    gr_mean = np.empty(0)
    t_m = np.empty(0)
    for i in range(index_shift, len(t)):
        gr_m = np.mean(gr[i-index_shift:i])
        gr_mean = np.append(gr_mean, gr_m)
        t_m = np.append(t_m, t[i])
    # plt.plot(t_m, gr_mean, '--', color='orange')


    # plt.hlines(np.mean(gr), np.min(t), np.max(t), color='blue')
    # gr1[np.argwhere(gr1<-1)]=0
    plt.plot(t1, gr1, label='David Grothrate', color='green')
    # indexes in 10time units
    time_in_1_index_shift = t1[-1] / len(t1)
    index_shift = int(time_units_to_average/time_in_1_index_shift)
    print('indext shift:',index_shift)
    gr_mean = np.empty(0)
    t1_m = np.empty(0)
    for i in range(index_shift, len(t1)):
        gr_m = np.mean(gr1[i-index_shift:i])
        gr_mean = np.append(gr_mean, gr_m)
        t1_m = np.append(t1_m, t1[i])
    plt.plot(t1_m, gr_mean, '--', color='green')
    # plt.hlines(np.mean(gr1), np.min(t1), np.max(t1), color='orange')

    # plt.vlines(64.218, -20, 2, color = 'red')
    plt.legend()
    
    plt.xlabel('simulation time, (Lref/cref)')
    if converged_value != None:
        plt.hlines(converged_value,0,t[-1], color='black')
    # plt.yscale('log')
    
    plt.ylim(*ylim)
    plt.xlim(*xlim)
    # fig.show()
    plt.show()

    

In [ ]:
def plot_Qes_history(nrg_path, title, tlim=None):
    species = parser_ts.read_species_names(parameters_path='/scratch/project_462000451/gene_out/gene_auto_97781/jet_97781_ion-scale/ex-jet_97781_ion-scale_batch-3/scanfiles0000/parameters')
    Qes, time = parser_ts.Qes_history(nrg_path)
    Qem, time = parser_ts.Qem_history(nrg_path)
    
    if tlim != None:
        tdif = np.abs(time-tlim)
        time = time[0:np.argmin(tdif)]
        Qes['1'] = Qes['1'][0:np.argmin(tdif)] 
    plt.figure()
    plt.title(title)

    plt.plot(time, Qes['1'], label=f'{species[0]}, Qes')
    print(Qes['1'])
    delta = Qes['1'] - np.roll(Qes['1'],1)
    
    # plt.plot(Qes['1'], label=f'{species[0]}, Qes', color='red')
    # plt.plot(delta, label=f'{species[0]}, Qes', color='red')
    
    # plt.xlim(0,200)
    # print(time[132])
    # print(delta[158])
    # print(Qes['1'][157])
    # print(Qes['1'][158])
    
    # plt.plot(time, Qes['2'], label=f'{species[1]}, Qes', color='orange')

    # plt.plot(time, Qem['1'], label=f'{species[0]}, Qem', color = 'blue')
    # plt.plot(time, Qem['2'], label=f'{species[1]}, Qem', color = 'cyan')
    plt.xlabel('simulation time, (Lref / cref)')
    plt.ylabel('Heat Flux')
    plt.yscale('log')
    plt.legend()
    plt.show()
    return time, Qes['1']

In [ ]:

from scipy.signal import find_peaks
def calculate_time_period(points):
    """
    Calculate the time_period of oscillations around a linear line given an array of points.
    
    Parameters:
    points (list of tuples): List of (x, y) points
    
    Returns:
    float: The time_period of the oscillations
    """
    # Convert points to numpy array for easier manipulation
    points = np.array(points)
    print('n points', len(points))
    # Fit a linear line to the points
    x = points[:, 0]
    y = points[:, 1]
    coefficients = np.polyfit(x, y, 1)
    linear_fit = np.polyval(coefficients, x)
    
    # Calculate the deviations from the linear fit
    deviations = y - linear_fit
    # print('linear_fit', linear_fit)
    # print('deviations',deviations)
    # Find peaks in the deviations
    peaks, _ = find_peaks(deviations)
    print('peaks,',x[peaks])
    num_peaks = len(peaks)
    # Calculate the distances between consecutive peaks
    peak_distances = np.diff(x[peaks])
    print('peak distances', peak_distances)
    
    # Calculate the average time_period
    if len(peak_distances) > 0:
        time_period = np.mean(peak_distances)
    else:
        time_period = None
    
    return time_period, peaks


In [ ]:
def calculate_growthrate_raw(scanfiles_dir, suffix, calc_from_apar=False):
    calc_from_apar=calc_from_apar
    suffix_index = int(suffix)-1
    suffix = '_'+suffix
    parameters_path = os.path.join(scanfiles_dir, f'parameters{suffix}')
    
    par = Parameters()
    par.Read_Pars(parameters_path)
    pars = par.pardict

    field = fieldfile(os.path.join(scanfiles_dir,'field'+suffix),pars)
    field.set_time(field.tfld[-1])
    imax = np.unravel_index(np.argmax(abs(field.phi()[:,0,:])),(field.nz,field.nx))
    phi = np.empty(0,dtype='complex128')
    if pars['n_fields'] > 1:
        imaxa = np.unravel_index(np.argmax(abs(field.apar()[:,0,:])),(field.nz,field.nx))
        apar = np.empty(0,dtype='complex128')

    time = np.empty(0)
    for i in range(len(field.tfld)):
        field.set_time(field.tfld[i])
        phi = np.append(phi,field.phi()[imax[0],0,imax[1]])
        if pars['n_fields'] > 1:
            apar = np.append(apar,field.apar()[imaxa[0],0,imaxa[1]])
        time = np.append(time,field.tfld[i])
    
    if len(phi) < 2.0:
        output_zeros = True
        omega = 0.0+0.0J
    else:
        output_zeros = False
        if calc_from_apar:
            print( "Calculating omega from apar")
            if pars['n_fields'] < 2:
                NotImplemented
                #stop
            omega = np.log(apar/np.roll(apar,1))
            dt = time - np.roll(time,1)
            omega /= dt
            omega = np.delete(omega,0)
            time = np.delete(time,0)
        else:
            omega = np.log(phi/np.roll(phi,1))
            dt = time - np.roll(time,1)
            omega /= dt
            omega = np.delete(omega,0)
            time = np.delete(time,0)

    gamma = np.real(omega)
    omega = np.imag(omega)
    
    return gamma, time

def Qes_history(scanfiles_dir, suffix, species_names=None):
    nrg_path=os.path.join(scanfiles_dir, f'nrg_{suffix}')
    time = []
    species_number=1
    Qes = {}
    with open(nrg_path, 'r') as nrg_file:
        # do loop once to initiate Qes dict
        for line in nrg_file:
            row = line.split(' ')
            row = [item for item in row if item != '']
            if len(row)==1:
                time.append(row[0])
                species_number=1
            else:
                if len(time)==1:
                    Qes[str(species_number)] = [row[6]]
                else:
                    Qes[str(species_number)].append(row[6])
                species_number += 1
    for i in range(1,species_number):
        Qes[str(i)] = np.array(Qes[str(i)]).astype('float')
    
    return Qes, np.array(time).astype('float')  

def correct_growthrate(gamma, gamma_time, Qes_e, Qes_e_time):
    deltaQ = Qes_e - np.roll(Qes_e, 1)
    ignore_times = Qes_e_time[np.argwhere(deltaQ < 1e-10)]
    Qes_e_corrected = Qes_e[np.argwhere(deltaQ < 1e-10)][:,0]
    Qes_e_time_corrected = Qes_e_time[np.argwhere(deltaQ < 1e-10)][:,0]
    
    # print('GAMMA TIME BEFORE CORRECTION',gamma_time)
    
    # Check the gamma near the ignore times and removes ones that have dropped.
    for t in ignore_times:
        tdif = np.abs(gamma_time-t)
        arg_t_closest = np.argmin(tdif)
        # sometimes the closes to the ignore time is not the blip, so we take the min within 5 steps away
        if arg_t_closest < 5:
            arg_delete = arg_t_closest + np.argmin(gamma[0: arg_t_closest+5])
        else:
            arg_delete = arg_t_closest - 5 + np.argmin(gamma[arg_t_closest-5: arg_t_closest+5])
        gamma = np.delete(gamma, arg_delete)
        gamma_time = np.delete(gamma_time, arg_delete)
    # print('GAMMA TIME AFTER CORRECTION',gamma_time)
    return gamma, gamma_time, Qes_e_corrected, Qes_e_time_corrected

def get_growthrate_corrected(scanfiles_dir, suffix):
    Qes, Q_time = Qes_history(scanfiles_dir, suffix)
    Qes_e = Qes['1'] # only if electrons are first in parameters file

    growthrate, g_time = calculate_growthrate_raw(scanfiles_dir, suffix)
    growthrate_corrected, time_corrected, Qes_e_corrected, Qes_e_time_corrected = correct_growthrate(growthrate, g_time, Qes_e, Q_time)
    return growthrate_corrected, time_corrected, Qes_e_corrected, Qes_e_time_corrected

In [ ]:
# %matplotlib ipympl
# %matplotlib widget
import sys
sys.path.append('/project/project_462000451/enchanted-surrogates_11feb2025_daniel/src')
sys.path.append('/project/project_462000451/enchanted-surrogates_11feb2025_daniel/submodules')
from IFS_scripts.fieldlib import fieldfile
from IFS_scripts.ParIO import Parameters

suffix = '0001'
dir = '/scratch/project_462000451/daniel/sprint_out/gene/full_test/1.0-1.8-3.0-4.4-0.05-0.1-0.5-2.8/1.6907219221235361-4.066386570204081-0.08957719078481746-0.08957719078481746-1.6580178585501517/scanfiles0065'#'/scratch/project_462000451/gene_out/gene_auto_97781/jet_97781_ion-scale_98/ex-jet_97781_ion-scale_98_batch-0/scanfiles0000'
plot_growthrate_history(dir, suffix)#, '', xlim=[0,50], ylim=[0,1.5])#, xlim=[175, 200], ylim=[0.536,0.537])
time, Qes = plot_Qes_history(nrg_path=os.path.join(dir,'nrg_'+suffix),title='title')
lQes = np.log(Qes)/10e30
# gr, gr_time, Qes, time = get_growthrate_corrected(dir, suffix)
# print('Q',Qes)
# print('lg Q',np.log(Qes))
points = np.load(os.path.join(dir, f't_eQes_for_window_calc_{suffix}.npy')).T
fig = plt.figure()
plt.plot(time, lQes)
# plt.plot(points[0], points[1], '.', color='red',label='time period calc points')
# plt.xlim(10,20)
# plt.ylim(-1,5)
# plt.xlim(87.6,87.8)
# plt.ylim(30,31)

tp, peaks = calculate_time_period(points.T)
plt.plot(points[0][peaks],points[1][peaks], '.', color='orange')
plt.xlabel('simulation time, cref/Lref')
plt.ylabel('Linear Electron Heat Flux')
plt.legend()
fig = plt.figure()
plt.plot(points[0], points[1], '.', color='red',label='time period calc points')
plt.plot(points[0][peaks],points[1][peaks], '.', color='orange')
plt.legend()
plt.xlabel('simulation time, cref/Lref')
plt.ylabel('Linear Electron Heat Flux')



# plt.plot(time[44],Qes[44], 'x', color = 'red')

NameError: name 'sys' is not defined

In [22]:
import pandas as pd
suffix_s = ['0001','0002']
df = pd.DataFrame(columns=['growthrate','stopped_early?','time_to_convergence'], index=suffix_s)

df.loc['0001', 'growthrate'] = 2
np.isnan(df.loc['0002', 'growthrate'])

True

In [34]:
dist_between_peaks = np.array([6.79098, 6.78132, 6.79098, 6.78132, 6.79098])
# m = np.mean(a)
# std = np.sqrt(np.var(a))

print(np.mean(dist_between_peaks), np.sqrt(np.var(dist_between_peaks)))

np.array(np.abs(dist_between_peaks-np.mean(dist_between_peaks)) > np.sqrt(np.var(dist_between_peaks)))

# m, std


6.787116 0.00473241418305721


array([False,  True, False,  True, False])